# MLPy — Sesión 6: Inicialización de Pesos y Funciones de Activación

En esta sesión estudiamos cómo **activaciones** e **inicialización** afectan:

- el flujo de activaciones (forward)
- el flujo de gradientes (backward)
- la estabilidad del entrenamiento

Mantendremos constantes:
- dataset
- arquitectura base
- optimizador


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt

torch.manual_seed(0)


## 1) Dataset simple (clasificación 2D)

Usamos el mismo dataset sintético para aislar efectos de activación/inicialización.


In [ ]:
N = 800
x0 = torch.randn(N//2, 2) + torch.tensor([-2.0, 0.0])
x1 = torch.randn(N//2, 2) + torch.tensor([ 2.0, 0.0])

X = torch.cat([x0, x1], dim=0)
y = torch.cat([torch.zeros(N//2), torch.ones(N//2)]).long()

# Mezclar
perm = torch.randperm(N)
X, y = X[perm], y[perm]

# Split entrenamiento / validación
train_frac = 0.8
n_train = int(N * train_frac)
X_train, y_train = X[:n_train], y[:n_train]
X_val,   y_val   = X[n_train:], y[n_train:]

train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
val_loader   = DataLoader(TensorDataset(X_val, y_val), batch_size=32, shuffle=False)

X_train.shape, X_val.shape


## 2) Funciones auxiliares (train/eval)


In [ ]:
criterion = nn.CrossEntropyLoss()

def accuracy_from_logits(logits, y):
    return (logits.argmax(dim=1) == y).float().mean().item()

def train_one_epoch(model, loader, optimizer):
    model.train()
    loss_sum, acc_sum, n = 0.0, 0.0, 0
    for xb, yb in loader:
        logits = model(xb)
        loss = criterion(logits, yb)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_sum += loss.item()
        acc_sum += accuracy_from_logits(logits, yb)
        n += 1
    return loss_sum / n, acc_sum / n

@torch.no_grad()
def eval_one_epoch(model, loader):
    model.eval()
    loss_sum, acc_sum, n = 0.0, 0.0, 0
    for xb, yb in loader:
        logits = model(xb)
        loss = criterion(logits, yb)
        loss_sum += loss.item()
        acc_sum += accuracy_from_logits(logits, yb)
        n += 1
    return loss_sum / n, acc_sum / n


## 3) Modelo con activación configurable


In [ ]:
def make_model(activation: str):
    activation = activation.lower().strip()
    if activation == "relu":
        act = nn.ReLU()
    elif activation == "tanh":
        act = nn.Tanh()
    elif activation == "sigmoid":
        act = nn.Sigmoid()
    elif activation == "leaky_relu":
        act = nn.LeakyReLU(0.01)
    else:
        raise ValueError(f"Activación desconocida: {activation}")

    return nn.Sequential(
        nn.Linear(2, 64),
        act,
        nn.Linear(64, 64),
        act,
        nn.Linear(64, 2)
    )


## 4) Inicialización de pesos


In [ ]:
def init_weights(model, scheme: str):
    scheme = scheme.lower().strip()
    for m in model.modules():
        if isinstance(m, nn.Linear):
            if scheme == "xavier":
                nn.init.xavier_normal_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif scheme == "he":
                nn.init.kaiming_normal_(m.weight, nonlinearity="relu")
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif scheme == "default":
                pass
            else:
                raise ValueError(f"Inicialización desconocida: {scheme}")


## 5) Experimento principal: activación × inicialización


In [ ]:
def run_experiment(activation: str, init_scheme: str, epochs: int = 30, lr: float = 0.1, seed: int = 0):
    torch.manual_seed(seed)
    model = make_model(activation)
    init_weights(model, init_scheme)
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)

    hist = {"train_loss": [], "val_loss": [], "train_acc": [], "val_acc": []}

    for _ in range(epochs):
        tr_loss, tr_acc = train_one_epoch(model, train_loader, optimizer)
        va_loss, va_acc = eval_one_epoch(model, val_loader)

        hist["train_loss"].append(tr_loss)
        hist["val_loss"].append(va_loss)
        hist["train_acc"].append(tr_acc)
        hist["val_acc"].append(va_acc)

    return hist

configs = [
    ("relu", "default"),
    ("relu", "he"),
    ("tanh", "xavier"),
    ("sigmoid", "xavier"),
    ("leaky_relu", "he"),
]

results = {f"{a}+{i}": run_experiment(a, i) for a, i in configs}
list(results.keys())


In [ ]:
plt.figure()
for label, hist in results.items():
    plt.plot(hist["val_loss"], label=label)
plt.xlabel("Época")
plt.ylabel("Val loss")
plt.legend()
plt.show()


In [ ]:
plt.figure()
for label, hist in results.items():
    plt.plot(hist["val_acc"], label=label)
plt.xlabel("Época")
plt.ylabel("Val accuracy")
plt.legend()
plt.show()


## 6) Diagnóstico interno: neuronas muertas (ReLU)

Medimos el porcentaje de activaciones **exactamente 0** después de ReLU.


In [ ]:
@torch.no_grad()
def relu_zero_fraction(model: nn.Module, loader) -> float:
    model.eval()
    zeros, total = 0, 0

    for xb, _ in loader:
        h = xb
        for layer in model:
            if isinstance(layer, nn.Linear):
                h = layer(h)
            elif isinstance(layer, nn.ReLU):
                h = layer(h)
                zeros += (h == 0).sum().item()
                total += h.numel()
            else:
                h = layer(h)

    return zeros / total if total > 0 else 0.0

relu_default = make_model("relu")
init_weights(relu_default, "default")

relu_he = make_model("relu")
init_weights(relu_he, "he")

relu_default_zero = relu_zero_fraction(relu_default, train_loader)
relu_he_zero = relu_zero_fraction(relu_he, train_loader)

relu_default_zero, relu_he_zero


## 7) (Opcional) Gradientes que desaparecen: norma de gradiente por capa

Medimos normas de gradiente en una sola iteración para ver si el gradiente se atenúa.


In [ ]:
def grad_norms_one_batch(model: nn.Module, xb: torch.Tensor, yb: torch.Tensor):
    model.train()
    logits = model(xb)
    loss = criterion(logits, yb)

    # limpiar grads
    for p in model.parameters():
        if p.grad is not None:
            p.grad.zero_()

    loss.backward()

    norms = []
    for name, p in model.named_parameters():
        if p.grad is not None:
            norms.append((name, float(p.grad.norm())))
    return float(loss.item()), norms

xb, yb = next(iter(train_loader))

m1 = make_model("relu"); init_weights(m1, "default")
m2 = make_model("relu"); init_weights(m2, "he")

loss1, norms1 = grad_norms_one_batch(m1, xb, yb)
loss2, norms2 = grad_norms_one_batch(m2, xb, yb)

loss1, loss2, norms1[:2], norms2[:2]


## 8) Preguntas de cierre

1. ¿Qué combinación mostró entrenamiento más estable en validación?

2. ¿Qué activaciones tienden a saturarse (y por qué eso afecta gradientes)?

3. ¿Qué observas al comparar ReLU con inicialización default vs He?

4. ¿Cambiar optimizador arreglaría un problema de inicialización? ¿por qué?

5. Si aumentamos profundidad, ¿qué esperas que ocurra con sigmoid/tanh?

